# Reproducing plots as interactive figures (w/Vizier)
## The Lyα Reference Sample. XIV. Lyα Imaging of 45 Low-redshift Star-forming Galaxies and Inferences on Global Emission
- Published 2023 May 9 • © 2023. The Author(s). Published by the American Astronomical Society.
- The Astrophysical Journal Supplement Series, Volume 269, Number 2
- Citation Jens Melinder et al 2023 ApJS 266 15
- [https://doi.org/10.3847/1538-4365/acc2b8](https://doi.org/10.3847/1538-4365/acc2b8)
- Vizier ingested all the tables from this article:   [J/ApJS/266/15](https://doi.org/10.26093/cds/vizier.22660015)
- And provides [notebooks](https://cdsarc.cds.unistra.fr/vizier/notebook.gml?source=J/ApJS/266/15) for downloading and interacting with the data.
- This reproduction starts with their original notebook.

In [1]:
# Access astronomical databases
from pyvo import registry  # version >=1.4.1 

# Moc and HEALPix tools
from mocpy import MOC

# Sky visualization
from ipyaladin import Aladin

# For plots
import matplotlib.pyplot as plt

In [2]:
# Plotly setup
# Tend to stick with "graphical objects" rather than express or Dash

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

--------------------------------------------------------

Modify the <a href='https://vizier.cds.unistra.fr/'>VizieR catalogue</a> name (variable <b>CATALOGUE</b>) and anything else you might feel like exploring!

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1

**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [3]:
# the catalogue name in VizieR
CATALOGUE = "J/ApJS/266/15"

We first retrieve catalogue information.

In [4]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [5]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

The Ly{alpha} Reference Sample (LARS). XIV.
Short Name: J/ApJS/266/15
IVOA Identifier: ivo://cds.vizier/j/apjs/266/15
Access modes: conesearch, tap#aux, web
Multi-capability service -- use get_service()

We present Ly{alpha} imaging of 45 low-redshift star-forming galaxies observed
with the Hubble Space Telescope. The galaxies have been selected to have
moderate to high star formation rates (SFRs) using far-ultraviolet (FUV)
luminosity and H{alpha} equivalent width criteria, but no constraints on
Ly{alpha} luminosity. We employ a pixel stellar continuum fitting code to
obtain accurate continuum-subtracted Ly{alpha}, H{alpha}, and H{beta} maps. We
find that Ly{alpha} is less concentrated than FUV and optical line emission in
almost all galaxies with significant Ly{alpha} emission. We present global
measurements of Ly{alpha} and other quantities measured in apertures designed
to capture all of the Ly{alpha} emission. We then show how the escape fraction
of Ly{alpha} relates to a number o

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [6]:
voresource.creators[0]

'Melinder J.'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.
> Note that Vizier merged Tables 1,2,4-7 into one large catalog, "galaxies"

In [7]:
tables = voresource.get_tables()
tables

{'J/ApJS/266/15/galaxies': <VODataServiceTable name="J/ApJS/266/15/galaxies">... 93 columns ...</VODataServiceTable>,
 'J/ApJS/266/15/table8': <VODataServiceTable name="J/ApJS/266/15/table8">... 23 columns ...</VODataServiceTable>}

In [8]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/ApJS/266/15/galaxies', 'J/ApJS/266/15/table8']

The actual data can then be access using any of the ``access_modes`` of the voresource.

In [9]:
voresource.access_modes()

{'conesearch', 'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [10]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/266/15'

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [11]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query 
# this also happens to be the same list of objects in Figure 2 of the article, LARS 001-004
tap_service = voresource.get_service("tap")
tap_records = voresource.get_service("tap").run_sync(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

<DALResultsTable length=10>
recno  Nsb    ID           SDSS         zspec  r_zspec  EWHa  ...      FUV                      Opt                 PapXI        SimbadName           _RA      _DE   
                                                       0.1 nm ...                                                                                     deg      deg   
int32 int16 object        object       float64   str1  int16  ...     object                   object               int16          object           float64  float64 
----- ----- ------ ------------------- ------- ------- ------ ... -------------- ---------------------------------- ----- ------------------------ --------- --------
    1    30 LARS01 J132844.05+435550.5   0.028            560 ... F140LP, F150LP UVIS/F336W, UVIS/F438W, UVIS/F775W     1 SDSS J132844.05+435550.5 202.18356 43.93069
    2    30 LARS02 J090704.88+532656.6    0.03            312 ... F140LP, F150LP UVIS/F336W, UVIS/F438W, UVIS/F775W     1 SDSS J090704.88+5326

In [12]:
tap_records.table.meta

{'ID': 'result_S1708437460671', 'name': 'result_S1708437460671'}

<!-- section position -->

We will explore the other access modes here.

### 3.2. Execute a cone search query

We use the Simple Cone Search (<a href='https://www.ivoa.net/documents/latest/ConeSearch.html'>SCS</a>) protocol of the virtual observatory (only available for tables with positions).

Adapt the radius and position of the center to your needs.

In [13]:
conesearch_radius =  90.  # in degrees
conesearch_center = (125.99121093749999, 28.101088898358825)

In [14]:
conesearch_records = voresource.get_service("conesearch").search(
    pos=conesearch_center,
    sr=conesearch_radius,
)
conesearch_records

<DALResultsTable length=44>
    _r    recno  Nsb     ID           SDSS         zspec  r_zspec  EWHa  ... PapXI Prof        SimbadName        NED  LEDA    _RA       _DE   
                                                                  0.1 nm ...                                                  deg       deg   
 float64  int32 int16   str7         str19        float32   str1  int16  ... int16 str4          str24           str3 str4  float64   float64 
--------- ----- ----- ------- ------------------- ------- ------- ------ ... ----- ---- ------------------------ ---- ---- --------- ---------
61.418225     1    30  LARS01 J132844.05+435550.5   0.028            560 ...     1 Prof SDSS J132844.05+435550.5  NED LEDA 202.18356  43.93069
26.561471     2    30  LARS02 J090704.88+532656.6   0.030            312 ...     1 Prof SDSS J090704.88+532656.6  NED LEDA 136.77036  53.44905
57.478833     3    30  LARS03 J131535.06+620728.6   0.031            238 ...     1 Prof SDSS J131535.06+620728.6  

## 4. Get the catalogue coverage

VizieR also provides the coverage of each catalogue with Multiple Order Coverage (<a href='https://ivoa.net/documents/MOC/'>MOC</a>) at order 10 -- meaning with a 0.001 rad spacial resolution.

or with ipyaladin

In [15]:
aladin = Aladin()
aladin

Aladin(options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'overlay_sur…

We can add the tables and coverage in this ipyaladin window.

In [16]:
conesearch = conesearch_records.to_table()
print(conesearch.meta)
aladin.add_table(conesearch)

{'ID': 'J_ApJS_266_15_galaxies', 'name': 'J/ApJS/266/15/galaxies', 'description': 'characteristics (Table 1), SDSS derived properties (Table 2), Size measurements (Table 4), Global Ly{alpha}, H{alpha}, and FUV measurements (Table 5), Derived properties (Table 6) and HST imaging for individual galaxies (Table 7)'}


In [17]:
fig2sources = tap_records.to_table()
print(fig2sources.meta)
aladin.add_table(fig2sources)

{'ID': 'result_S1708437460671', 'name': 'result_S1708437460671'}


Make some plots. Let's try Figure 13